# Title

#### Explain

Importamos librerías

In [59]:
from pymongo import MongoClient
import pandas as pd
#from src.gameslib import 
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()
import re

Indicamos la base de datos de MongoDB (en carpeta /output, unzip y cargar en servidor local mongodb)

In [60]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

buscamos las empresas de videojuegos, diseño, etc

In [61]:
query = {"$or": [{"category_code":"games_video"},{"tag_list":{'$regex':'/design|game|e-sport/'}}]}
select = {"name":1,"category_code":1,"number_of_employees":1,"tag_list":1,"offices":1, "_id":0}

designers = list(db["companies"].find(query,select))


In [77]:

df = pd.DataFrame(designers)
df.head()

,name,category_code,number_of_employees,tag_list,offices
0,Flektor,games_video,NaN,"flektor, photo, video","[{'description': None, 'address1': '8536 Natio..."
1,Lala,games_video,NaN,,"[{'description': 'Lala Headquarters', 'address..."
2,Joost,games_video,0.0,"iptv, babelgum, television, video, thevenicepr...","[{'description': '', 'address1': '100 5th Ave ..."
3,Babelgum,games_video,NaN,"iptv, web2ireland","[{'description': '', 'address1': '', 'address2..."
4,Sparter,games_video,NaN,"gaming, game, wow, worldofwarcraft, virtualgoods","[{'description': None, 'address1': None, 'addr..."


Como vemos arriba, hay que *desmembrar* la columna de offices para sacar de la lista de diccionarios las coordenadas de las distintas ubicaciones

In [78]:
#explode and expand
df = df.explode('offices')
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
cleanData = pd.concat([df,dfOfficeData], axis=1)

Drop NaN in coords

In [79]:
cleanData.dropna(subset=['latitude','longitude'],inplace=True)
cleanData.reset_index(inplace=True,drop=True)
cleanData.drop(columns=["offices"])

cleanData.fillna(value={'number_of_employees':5.0},inplace=True)
cleanData

,name,category_code,number_of_employees,tag_list,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Flektor,games_video,5.0,"flektor, photo, video","{'description': None, 'address1': '8536 Nation...",None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
1,Lala,games_video,5.0,,"{'description': 'Lala Headquarters', 'address1...",Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369
2,Joost,games_video,0.0,"iptv, babelgum, television, video, thevenicepr...","{'description': '', 'address1': '100 5th Ave F...",,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,40.746497,-74.009447
3,Babelgum,games_video,5.0,"iptv, web2ireland","{'description': '', 'address1': '', 'address2'...",,,,,London,None,GBR,53.344104,-6.267494
4,Sparter,games_video,5.0,"gaming, game, wow, worldofwarcraft, virtualgoods","{'description': None, 'address1': None, 'addre...",None,None,None,None,None,None,USA,37.090240,-95.712891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,Fictionaut,games_video,2.0,"fiction, writers, literary, publishing, commun...","{'description': 'New York', 'address1': '32-14...",New York,32-14 42nd St,,11103,Astoria,NY,USA,40.759795,-73.916127
761,Fictionaut,games_video,2.0,"fiction, writers, literary, publishing, commun...","{'description': 'Houston', 'address1': '2208 C...",Houston,2208 Colquitt,,77098,Houston,TX,USA,29.735462,-95.413243
762,Tiny Speck,games_video,45.0,,"{'description': '', 'address1': '163 Hastings ...",,163 Hastings St W,,BC V6B 1H5,Vancouver,None,CAN,49.282455,-123.109217
763,My TV Shows,games_video,2.0,"tv, shows, episodes","{'description': 'Lisbon Office', 'address1': '...",Lisbon Office,Rua RosÃ¡lia de Castro,,1700,Lisbon,None,PRT,38.752367,-9.146221


Con la longitud y latitud limpias y sin nulos, creamos una columna tipo POINT para cartoframe

In [80]:
def toCartoPoint(lon,lat):
    return f'POINT ({str(lon)} {str(lat)})'

toCartoPoint(-0.116537,51.517742)

'POINT (-0.116537 51.517742)'

In [81]:
cleanData["geometry"] = cleanData[["latitude","longitude"]].apply(lambda x:toCartoPoint(x.longitude,x.latitude), axis=1)
cleanData[["latitude","longitude","geometry"]].head()

,latitude,longitude,geometry
0,34.025958,-118.379768,POINT (-118.379768 34.025958)
1,37.451151,-122.154369,POINT (-122.154369 37.451151)
2,40.746497,-74.009447,POINT (-74.0094471 40.7464969)
3,53.344104,-6.267494,POINT (-6.267494 53.344104)
4,37.090240,-95.712891,POINT (-95.712891 37.09024)


In [ ]:
import geopandas as gpd

from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import size_continuous_layer
from cartoframes.viz.widgets import histogram_widget

In [82]:
Map(Layer(cleanData))

In [83]:
Map(size_continuous_layer(cleanData, 'number_of_employees'))

In [ ]:
'''import math
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
cleanData[["latitude","longitude","location"]]
'''

In [6]:
lat='38.345887'
long='-0.490576'
tipositio='school'
radio='2500'
API_KEY = os.getenv("PLACES_API_TOKEN")
URL = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{long}&type={tipositio}&radius={radio}&key={API_KEY}'


In [25]:
response_data = requests.get(URL)


In [8]:
data = len(list(response_data))
data

254

In [26]:
json_data = json.loads(response_data.text)
lista_coles = json_data['results']
for i in range(len(lista_coles)):
    print(lista_coles[i]['geometry']['location'])

{'lat': 38.3567087, 'lng': -0.5058289}
{'lat': 38.34958, 'lng': -0.4740800000000001}
{'lat': 38.3523635, 'lng': -0.5051052}
{'lat': 38.34316400000001, 'lng': -0.492161}
{'lat': 38.3619991, 'lng': -0.4909873000000001}
{'lat': 38.3534778, 'lng': -0.4856677}
{'lat': 38.3466503, 'lng': -0.5062078999999999}
{'lat': 38.3392515, 'lng': -0.5077184}
{'lat': 38.3553565, 'lng': -0.4739583999999999}
{'lat': 38.3447153, 'lng': -0.4900758}
{'lat': 38.345251, 'lng': -0.491248}
{'lat': 38.3411174, 'lng': -0.4958768}
{'lat': 38.3432417, 'lng': -0.4840492}
{'lat': 38.35493140000001, 'lng': -0.4869952999999999}
{'lat': 38.3400287, 'lng': -0.4991959}
{'lat': 38.35236520000001, 'lng': -0.5032013}
{'lat': 38.3534297, 'lng': -0.4964602}
{'lat': 38.34400730000001, 'lng': -0.4859191}
{'lat': 38.35245219999999, 'lng': -0.4748248999999999}
{'lat': 38.3486923, 'lng': -0.4747926}


In [15]:
# From Response/JSON to dict:
# data_dict = json.loads(response_data.text)
# From dict to json:
# y = json.dumps(data_dict)



In [46]:
lat='38.345887'
long='-0.490576'
radio='1000'
placename = 'Starbucks'
URL = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={placename}&inputtype=textquery&fields=formatted_address,name&locationbias=circle:{radio}@{lat},{long}&key={API_KEY}'

In [47]:
response_data = requests.get(URL)


In [48]:
json_data = json.loads(response_data.text)
#starb = json_data['results']
json_data

{'candidates': [{'formatted_address': 'Av. Maisonnave, 53, 03003 Alicante, España',
   'name': 'Starbucks'}],
 'status': 'OK'}